In [84]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn import feature_extraction, linear_model, model_selection, metrics
from sklearn import ensemble
from scipy import sparse
import sys
sys.path.append("..")
from src.make_model_lstm import hate_speech_model

In [85]:
#watch overfitting


In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
df = pd.read_csv('../data/labeled_data.csv')

In [88]:
tweets = df['tweet'].apply(lambda x: x.lstrip('!'))

In [89]:
y = np.argmax(df[['hate_speech', 'offensive_language', 'neither']].values, axis = 1)

In [90]:
stop_words = set(stopwords.words('english')) 

In [91]:
token = nltk.tokenize.casual.TweetTokenizer()

In [92]:
lemmatizer = WordNetLemmatizer()

In [93]:
cleaned = []
wordDict = {}
i = 1
for tweet in tweets:
    tokenized = token.tokenize(tweet)
    newSent = []
    for word in tokenized:
        if word not in stop_words:
            newWord = lemmatizer.lemmatize(word)
            if newWord not in wordDict:
                wordDict[newWord] = i
                i += 1
            newSent.append(newWord)
            
    cleaned.append(newSent)

In [94]:
print(cleaned[0])
print(len(cleaned))

['RT', '@mayasolovely', ':', 'As', 'woman', 'complain', 'cleaning', 'house', '.', '&', 'man', 'always', 'take', 'trash', '...']
24783


In [95]:
maxWords = max(map(len, cleaned))

In [96]:
def create_seq(sent, vocab, maxWords):
    n = len(sent)
    numZeros = maxWords - n
    result = [0]*numZeros
    
    for word in sent:
        if word in vocab:
            result.append(vocab[word] + 2)
        else:
            result.append(1)
            
    return result

In [97]:
sequences = np.array(list(map(lambda x: create_seq(x, wordDict, maxWords), cleaned)))

In [98]:
sequences[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [99]:
y_cat = np.zeros((len(y), 3))
for i, l in enumerate(y):
    y_cat[i, l] = 1

In [100]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(sequences, y_cat, test_size = .1)

In [101]:
print(max(wordDict.values()))

41243


In [154]:
model = hate_speech_model()
model.build_model()
model.fit(X_train, y_train, epochs = 100, class_weight = {0: 1, 1: .05, 2: .15})
preds = model.predict(X_test)

>> Compiled...
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 21188 samples, validate on 1116 samples
Epoch 1/100
21188/21188 [==============================] - 19s 889us/sample - loss: 0.1341 - val_loss: 0.1287
Epoch 2/100
21188/21188 [==============================] - 15s 722us/sample - loss: 0.1333 - val_loss: 0.1281
Epoch 3/100
21188/21188 [==============================] - 15s 710us/sample - loss: 0.1324 - val_loss: 0.1273
Epoch 4/100
21188/21188 [==============================] - 15s 714us/sample - loss: 0.1313 - val_loss: 0.1263
Epoch 5/100
21188/21188 [==============================] - 15s 712us/sample - loss: 0.1299 - val_loss: 0.1251
Epoch 6/100
21188/21188 [==============================] - 15s 716us/sample - loss: 0.1283 - val_loss: 0.1239
Epoch 7/100
21188/21188 [==============================] - 15s 712us/sample - loss: 0.1272 - val_loss: 0.1235
Epoch 8/100
21188/21188 [==============================] - 15s 705us/sample - loss: 0.1268 - val_loss: 0.1234
Epoch 

In [155]:
print(metrics.roc_auc_score(y_test, preds))

0.8694688863729785


In [156]:
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(preds, axis = 1)

In [157]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.14      0.74      0.24       132
           1       0.95      0.70      0.80      1959
           2       0.71      0.61      0.66       388

    accuracy                           0.69      2479
   macro avg       0.60      0.68      0.57      2479
weighted avg       0.87      0.69      0.75      2479



In [158]:
print(metrics.confusion_matrix(y_true, y_pred))

[[  98   22   12]
 [ 504 1371   84]
 [  97   55  236]]
